In [1]:
# importation des bibliothèques 
import requests
import pandas as pd

## Récupérons les données de Paris en temps réel

In [2]:

# URL de l'API
url = "https://opendata.paris.fr/api/explore/v2.1/catalog/datasets/velib-disponibilite-en-temps-reel/records?"

# Fonction pour collecter toutes les données depuis l'API
def collect_velib_data(url):
    velib_data = []  # Initialiser une liste pour stocker les données
    start = 0  # Point de départ pour la pagination
    rows_per_page = 100  # Nombre de lignes par page, ajustable si nécessaire

    # Boucle pour paginer et collecter les données
    while True:
        try:
            # Faire une requête API avec des paramètres de pagination
            response = requests.get(url, params={"start": start, "rows": rows_per_page})
            
            # Vérifier si la requête est réussie
            if response.status_code == 200:
                data = response.json()
                
                # Vérifier si la clé "results" est dans la réponse
                if "results" in data:
                    # Boucler sur chaque station dans "results"
                    for record in data["results"]:
                        # Initialiser les valeurs par défaut pour éviter les erreurs
                        coordonnees_geo = record.get("coordonnees_geo") or {}
                        
                        # Extraire les champs pertinents pour chaque station
                        station_info = {
                            "stationcode": record.get("stationcode"),
                            "name": record.get("name"),
                            "is_installed": record.get("is_installed"),
                            "capacity": record.get("capacity"),
                            "nombre_places_disponibles": record.get("numdocksavailable"),
                            "nombre_velos_disponibles": record.get("numbikesavailable"),
                            "mechanical": record.get("mechanical"),
                            "ebike": record.get("ebike"),
                            "is_renting": record.get("is_renting"),
                            "is_returning": record.get("is_returning"),
                            "duedate": record.get("duedate"),
                            "longitude": coordonnees_geo.get("lon"),
                            "latitude": coordonnees_geo.get("lat"),
                            "arrondissement_commune": record.get("nom_arrondissement_communes"),
                            "code_insee_commune": record.get("code_insee_commune")
                        }
                        
                        # Ajouter les informations de la station à la liste
                        velib_data.append(station_info)
                    
                    # Mettre à jour le point de départ pour la page suivante
                    start += rows_per_page

                    # Si moins de données sont retournées que `rows_per_page`, on est à la fin des données
                    if len(data["results"]) < rows_per_page:
                        break
                else:
                    print("Erreur : clé 'results' non trouvée dans la réponse.")
                    break
            else:
                print(f"Erreur lors de la requête API : {response.status_code}")
                break
        
        except ValueError:
            print("Erreur dans le décodage du JSON.")
            break

    return velib_data

# Collecter les données des stations
velib_data = collect_velib_data(url)

# Si des données sont récupérées, les convertir en DataFrame pour affichage
if velib_data:
    dfParis = pd.DataFrame(velib_data)
    dfParis.head()  # Afficher les données sous forme de tableau
else:
    print("Aucune donnée n'a été récupérée.")

dfParis.head()
#dfParis.shape

,stationcode,name,is_installed,capacity,nombre_places_disponibles,nombre_velos_disponibles,mechanical,ebike,is_renting,is_returning,duedate,longitude,latitude,arrondissement_commune,code_insee_commune
0,44015,Rouget de L'isle - Watteau,OUI,20,10,10,2,8,OUI,OUI,2024-12-09T15:28:22+00:00,2.396302,48.778193,Vitry-sur-Seine,94081
1,9020,Toudouze - Clauzel,OUI,21,16,4,0,4,OUI,OUI,2024-12-09T15:30:51+00:00,2.337360,48.879296,Paris,75056
2,14014,Jourdan - Stade Charléty,OUI,60,33,24,12,12,OUI,OUI,2024-12-09T15:28:14+00:00,2.343335,48.819428,Paris,75056
3,32017,Basilique,OUI,22,3,16,11,5,OUI,OUI,2024-12-09T15:34:01+00:00,2.358867,48.936269,Saint-Denis,93066
4,13007,Le Brun - Gobelins,OUI,48,39,7,4,3,OUI,OUI,2024-12-09T15:31:56+00:00,2.353468,48.835093,Paris,75056


In [3]:
dfParis.shape

(1486, 15)

In [4]:
# Ajouter une colonne vide
dfParis['ville'] = 'Paris'

In [5]:
dfParis.head()

,stationcode,name,is_installed,capacity,nombre_places_disponibles,nombre_velos_disponibles,mechanical,ebike,is_renting,is_returning,duedate,longitude,latitude,arrondissement_commune,code_insee_commune,ville
0,44015,Rouget de L'isle - Watteau,OUI,20,10,10,2,8,OUI,OUI,2024-12-09T15:28:22+00:00,2.396302,48.778193,Vitry-sur-Seine,94081,Paris
1,9020,Toudouze - Clauzel,OUI,21,16,4,0,4,OUI,OUI,2024-12-09T15:30:51+00:00,2.337360,48.879296,Paris,75056,Paris
2,14014,Jourdan - Stade Charléty,OUI,60,33,24,12,12,OUI,OUI,2024-12-09T15:28:14+00:00,2.343335,48.819428,Paris,75056,Paris
3,32017,Basilique,OUI,22,3,16,11,5,OUI,OUI,2024-12-09T15:34:01+00:00,2.358867,48.936269,Saint-Denis,93066,Paris
4,13007,Le Brun - Gobelins,OUI,48,39,7,4,3,OUI,OUI,2024-12-09T15:31:56+00:00,2.353468,48.835093,Paris,75056,Paris


## Récupérons les données de Toulouse en temps réel

In [6]:

# URL de l'API pour Toulouse
url = "https://data.toulouse-metropole.fr/api/explore/v2.1/catalog/datasets/stationnement-velo/records?limit=20"

# Fonction pour collecter toutes les données depuis l'API
def collect_bike_data_toulouse(url):
    bike_data = []  # Initialiser une liste pour stocker les données
    start = 0  # Point de départ pour la pagination
    rows_per_page = 20  # Nombre de lignes par page, ajustable si nécessaire

    # Boucle pour paginer et collecter les données
    while True:
        try:
            # Faire une requête API avec des paramètres de pagination
            response = requests.get(url, params={"start": start, "rows": rows_per_page})
            
            # Vérifier si la requête est réussie
            if response.status_code == 200:
                data = response.json()
                
                # Vérifier si la clé "results" est dans la réponse
                if "results" in data:
                    # Boucler sur chaque station dans "records"
                    for record in data["results"]:
                        # Extraire les champs pertinents pour chaque station
                        station_info = {
                            "latitude": record["geo_point_2d"]["lat"],
                            "longitude": record["geo_point_2d"]["lon"],
                            "lib_voie": record.get("lib_voie"),
                            "motdir": record.get("motdir"),
                            "no": record.get("no"),
                            "arrondissement_commune":record.get("commune"),
                            "insee": record.get("insee"),
                            "nombre_places_disponibles": record.get("nb_places"),
                            "coord_x": record.get("coord_x"),
                            "coord_y": record.get("coord_y")
                        }
                        
                        # Ajouter les informations de la station à la liste
                        bike_data.append(station_info)
                    
                    # Mettre à jour le point de départ pour la page suivante
                    start += rows_per_page

                    # Si moins de données sont retournées que `rows_per_page`, on est à la fin des données
                    if len(data["results"]) < rows_per_page:
                        break
                else:
                    print("Erreur : clé 'records' non trouvée dans la réponse.")
                    break
            else:
                print(f"Erreur lors de la requête API : {response.status_code}")
                break
        
        except ValueError:
            print("Erreur dans le décodage du JSON.")
            break

    return bike_data

# Collecter les données des stations
bike_data_toulouse = collect_bike_data_toulouse(url)

# Si des données sont récupérées, les convertir en DataFrame pour affichage
if bike_data_toulouse:
    dfToulouse = pd.DataFrame(bike_data_toulouse)
    #display(dfToulouse.head(15))  # Afficher les données sous forme de tableau
else:
    print("Aucune donnée n'a été récupérée.")


dfToulouse.head(15)
dfToulouse.shape

(1341, 10)

In [7]:
dfToulouse['ville'] = 'Toulouse'

In [8]:
#dfToulouse = dfToulouse.rename(columns={'commune': 'nom_arrondissement_communes'})

In [9]:
dfToulouse.head()

,latitude,longitude,lib_voie,motdir,no,arrondissement_commune,insee,nombre_places_disponibles,coord_x,coord_y,ville
0,43.596179,1.453828,ALL PAUL SABATIER,SABATIER - 0002 ALL PAUL SABATIER,2,TOULOUSE,31555,10,1.453828,43.596179,Toulouse
1,43.592579,1.448270,RUE ALFRED DUMERIL,DUMERIL (RUE ALFRED DUMERIL),None,TOULOUSE,31555,12,1.448270,43.592579,Toulouse
2,43.592382,1.449007,RUE ALFRED DUMERIL,DUMERIL (RUE ALFRED DUMERIL),None,TOULOUSE,31555,10,1.449007,43.592382,Toulouse
3,43.606585,1.438509,RUE DES PUITS CREUSES,CREUSES (RUE DES PUITS CREUSES),2,TOULOUSE,31555,10,1.438509,43.606585,Toulouse
4,43.608423,1.477831,RUE CHARLES GARNIER,GARNIER (RUE CHARLES GARNIER),12,TOULOUSE,31555,4,1.477831,43.608423,Toulouse


In [10]:
# Ajoutons la colonne 'nombre_velos_disponibles'
dfToulouse['nombre_velos_disponibles'] = dfToulouse['nombre_places_disponibles'].median()

## Récupérons les données de Lille en temps réel

In [11]:
# URL de l'API pour Lille
url = "https://data.lillemetropole.fr/data/ogcapi/collections/vlille_temps_reel/items?"

# Fonction pour collecter toutes les données depuis l'API
def collect_bike_data_lille(url, max_requests=5):
    bike_data = []  # Initialiser une liste pour stocker les données
    start = 0  # Point de départ pour la pagination
    rows_per_page = 300  # Nombre de lignes par page 

    # Boucle pour paginer et collecter les données
    for request_count in range(max_requests):
        try:
            # Faire une requête API avec des paramètres de pagination et un timeout
            response = requests.get(url, params={"start": start, "limit": rows_per_page}, timeout=10)
            
            # Vérifier si la requête est réussie
            if response.status_code == 200:
                try:
                    data = response.json()
                
                    # Vérifier si la clé "features" est dans la réponse
                    if "features" in data:
                        # Boucler sur chaque station dans "features"
                        for record in data["features"]:
                            properties = record.get("properties", {})
                            geometry = record.get("geometry", {})
        
                            # Extraire les champs pertinents pour chaque station
                            station_info = {
                                "nom": properties.get("nom"),
                                "adresse": properties.get("adresse"),
                                "code_insee": properties.get("code_insee"),
                                "arrondissement_commune": properties.get("commune"),
                                "etat": properties.get("etat"),
                                "type": properties.get("type"),
                                "nombre_places_disponibles": properties.get("nb_places_dispo"),
                                "nombre_velos_disponibles": properties.get("nb_velos_dispo"),
                                "etat_connexion": properties.get("etat_connexion"),
                                "longitude": geometry.get("coordinates", [None, None])[0],
                                "latitude": geometry.get("coordinates", [None, None])[1],
                                "date_modification": properties.get("date_modification")
                            }
                                
                            # Ajouter les informations de la station à la liste
                            bike_data.append(station_info)
                            
                        # Si moins de données sont retournées que `rows_per_page`, on est à la fin des données
                        if len(data["features"]) < rows_per_page:
                            break
                        else:
                            start += rows_per_page
                    else:
                        print("Erreur : clé 'features' non trouvée dans la réponse.")
                        break
                except ValueError:
                    print("Erreur lors du décodage du JSON.")
                    break
            else:
                print(f"Erreur lors de la requête API : {response.status_code}")
                break
            
        except (requests.Timeout, requests.RequestException) as e:
            print(f"Erreur lors de la requête : {e}")
            break

    return bike_data

# Collecter les données des stations avec un nombre de requêtes maximum limité
bike_data_lille = collect_bike_data_lille(url)

# Si des données sont récupérées, les convertir en DataFrame pour affichage
if bike_data_lille:
    dfLille = pd.DataFrame(bike_data_lille)
    display(dfLille.head(15))  # Afficher les données sous forme de tableau
else:
    print("Aucune donnée n'a été récupérée.")

# Afficher la taille et les premières lignes du DataFrame
dfLille.head()



,nom,adresse,code_insee,arrondissement_commune,etat,type,nombre_places_disponibles,nombre_velos_disponibles,etat_connexion,longitude,latitude,date_modification
0,METROPOLE EUROPEENNE DE LILLE,MEL RUE DU BALLON,None,LILLE,RÉFORMÉ,AVEC TPE,0,0,DÉCONNECTÉ,3.075992,50.641926,2022-11-29T10:47:16.181+00:00
1,UNIVERSITE CATHOLIQUE,64 Boulevard Vauban,None,Lille,EN SERVICE,AVEC TPE,31,0,CONNECTÉ,3.046134,50.632233,2024-12-09T19:31:15.251+00:00
2,JARDIN VAUBAN,19 Boulevard Vauban,None,Lille,EN SERVICE,AVEC TPE,18,0,CONNECTÉ,3.050447,50.636093,2024-12-09T19:31:15.252+00:00
3,MAISON FOLIE WAZEMMES,11 Rue de l'Hôpital Saint-Roch,None,Lille,EN SERVICE,AVEC TPE,5,15,CONNECTÉ,3.048961,50.624695,2024-12-09T19:31:15.253+00:00
4,MASSENA,87 Rue Masséna,None,Lille,EN SERVICE,AVEC TPE,13,27,CONNECTÉ,3.054738,50.631560,2024-12-09T19:31:15.253+00:00
5,REPUBLIQUE BEAUX ARTS,4 Rue d'Inkermann,None,Lille,EN SERVICE,AVEC TPE,14,10,CONNECTÉ,3.060299,50.630943,2024-12-09T19:31:15.254+00:00
6,PLACE DE STRASBOURG,7 Place de Strasbourg,None,Lille,EN SERVICE,AVEC TPE,16,0,CONNECTÉ,3.055307,50.633728,2024-12-09T19:31:15.254+00:00
7,PLACE RICHEBE,3 place Richebé,None,Lille,EN SERVICE,AVEC TPE,37,3,CONNECTÉ,3.062313,50.632366,2024-12-09T19:31:15.255+00:00
8,RUE DE TOUL,18 rue de Toul,None,Lille,EN SERVICE,AVEC TPE,36,0,CONNECTÉ,3.044986,50.633930,2024-12-09T19:31:15.255+00:00
9,RIHOUR,28 place Rihour,None,Lille,EN SERVICE,AVEC TPE,29,2,CONNECTÉ,3.062471,50.635890,2024-12-09T19:31:15.256+00:00


,nom,adresse,code_insee,arrondissement_commune,etat,type,nombre_places_disponibles,nombre_velos_disponibles,etat_connexion,longitude,latitude,date_modification
0,METROPOLE EUROPEENNE DE LILLE,MEL RUE DU BALLON,None,LILLE,RÉFORMÉ,AVEC TPE,0,0,DÉCONNECTÉ,3.075992,50.641926,2022-11-29T10:47:16.181+00:00
1,UNIVERSITE CATHOLIQUE,64 Boulevard Vauban,None,Lille,EN SERVICE,AVEC TPE,31,0,CONNECTÉ,3.046134,50.632233,2024-12-09T19:31:15.251+00:00
2,JARDIN VAUBAN,19 Boulevard Vauban,None,Lille,EN SERVICE,AVEC TPE,18,0,CONNECTÉ,3.050447,50.636093,2024-12-09T19:31:15.252+00:00
3,MAISON FOLIE WAZEMMES,11 Rue de l'Hôpital Saint-Roch,None,Lille,EN SERVICE,AVEC TPE,5,15,CONNECTÉ,3.048961,50.624695,2024-12-09T19:31:15.253+00:00
4,MASSENA,87 Rue Masséna,None,Lille,EN SERVICE,AVEC TPE,13,27,CONNECTÉ,3.054738,50.631560,2024-12-09T19:31:15.253+00:00


In [12]:
#doublons_lignes.head()

In [13]:
print(dfLille.shape)

(289, 12)


In [14]:
dfLille['ville'] = 'Lille'

In [15]:
# dfLille = dfLille.rename(columns={'commune': 'nom_arrondissement_communes'})

 #  Après avoir développé des algorithmes pour la collecte, passons à présent au nettoyage et la transformation des données (par exemple, mise au format des dates, traitement des données manquantes, etc.) pour chaque DataFrame.

In [16]:
# pip install sqlalchemy

In [17]:
from sqlalchemy import create_engine

# Étape 1 : Nettoyage et Concaténation des Données
# Étape 1 : Nettoyage et Concaténation des Données
def prepare_data_for_sql(dataframes, common_columns):

    # Filtrer les colonnes communes
    filtered_dfs = [df[common_columns] for df in dataframes]
    
    # Concaténer les DataFrames filtrés
    combined_data = pd.concat(filtered_dfs, ignore_index=True)
    
    return combined_data

# Colonnes communes à conserver
common_columns = ['arrondissement_commune', 'longitude', 'latitude', 
                  'nombre_places_disponibles', 'nombre_velos_disponibles', 'ville']

# Appel de la fonction
dataframes = [dfParis, dfToulouse, dfLille]
final_data = prepare_data_for_sql(dataframes, common_columns)

# Affichage du résultat pour vérification
final_data.head()


,arrondissement_commune,longitude,latitude,nombre_places_disponibles,nombre_velos_disponibles,ville
0,Vitry-sur-Seine,2.396302,48.778193,10,10.0,Paris
1,Paris,2.337360,48.879296,16,4.0,Paris
2,Paris,2.343335,48.819428,33,24.0,Paris
3,Saint-Denis,2.358867,48.936269,3,16.0,Paris
4,Paris,2.353468,48.835093,39,7.0,Paris


In [18]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3116 entries, 0 to 3115
Data columns (total 6 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   arrondissement_commune     3116 non-null   object 
 1   longitude                  3116 non-null   float64
 2   latitude                   3116 non-null   float64
 3   nombre_places_disponibles  3116 non-null   int64  
 4   nombre_velos_disponibles   3116 non-null   float64
 5   ville                      3116 non-null   object 
dtypes: float64(3), int64(1), object(2)
memory usage: 146.2+ KB


In [19]:
final_data['ville'].unique()

array(['Paris', 'Toulouse', 'Lille'], dtype=object)

In [20]:
clean_data = final_data

In [21]:
clean_data.head()

,arrondissement_commune,longitude,latitude,nombre_places_disponibles,nombre_velos_disponibles,ville
0,Vitry-sur-Seine,2.396302,48.778193,10,10.0,Paris
1,Paris,2.337360,48.879296,16,4.0,Paris
2,Paris,2.343335,48.819428,33,24.0,Paris
3,Saint-Denis,2.358867,48.936269,3,16.0,Paris
4,Paris,2.353468,48.835093,39,7.0,Paris


In [22]:
clean_data['ville'].unique()

array(['Paris', 'Toulouse', 'Lille'], dtype=object)

In [23]:
import sqlite3   

# Créer une connexion à la base de données 
conn = sqlite3.connect('info_velos.db') 

# Stocker les données de la table 'clean_data' dans la base de données SQL
clean_data.to_sql('clean_data', conn, if_exists='replace', index=False)

# Fermer la connexion
conn.close()

print("Les données ont été stockées dans la base de données avec succès.")


Les données ont été stockées dans la base de données avec succès.


In [24]:

# Connexion à la base de données SQLite
conn = sqlite3.connect('info_velos.db')
cursor = conn.cursor()

# Obtenir la liste des tables dans la base de données
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
tables = cursor.fetchall()

# Afficher les noms des tables
print("Tables dans la base de données :")
for table in tables:
    print(table[0])

conn.close()


Tables dans la base de données :
clean_data


In [25]:

# Connexion à la base de données SQLite
conn = sqlite3.connect('info_velos.db')
cursor = conn.cursor()

# Vérifier les premières lignes de la table 'clean_data'
cursor.execute("SELECT * FROM clean_data LIMIT 5;")
rows = cursor.fetchall()

# Afficher les premières lignes
print("Premières lignes de la table 'clean_data' :")
for row in rows:
    print(row)

conn.close()


Premières lignes de la table 'clean_data' :
('Vitry-sur-Seine', 2.3963020229163, 48.778192750803, 10, 10.0, 'Paris')
('Paris', 2.3373600840568547, 48.87929591733507, 16, 4.0, 'Paris')
('Paris', 2.3433353751898, 48.819428333369, 33, 24.0, 'Paris')
('Saint-Denis', 2.3588666820200914, 48.93626891059109, 3, 16.0, 'Paris')
('Paris', 2.3534681351338, 48.835092787824, 39, 7.0, 'Paris')


In [26]:
# Connexion à la base de données SQLite
conn = sqlite3.connect('info_velos.db')
cursor = conn.cursor()

# Obtenir les informations sur la structure de la table 'clean_data'
cursor.execute("PRAGMA table_info(clean_data);")
columns = cursor.fetchall()

# Afficher les informations sur les colonnes
print("Structure de la table 'clean_data' :")
for column in columns:
    print(column)

conn.close()


Structure de la table 'clean_data' :
(0, 'arrondissement_commune', 'TEXT', 0, None, 0)
(1, 'longitude', 'REAL', 0, None, 0)
(2, 'latitude', 'REAL', 0, None, 0)
(3, 'nombre_places_disponibles', 'INTEGER', 0, None, 0)
(4, 'nombre_velos_disponibles', 'REAL', 0, None, 0)
(5, 'ville', 'TEXT', 0, None, 0)


In [27]:
# !pip install folium pandas


In [28]:
# Se connecter à la base de données SQLite
conn = sqlite3.connect('info_velos.db')

# Charger les données des trois villes (Paris, Lille, Toulouse) dans un DataFrame
query = "SELECT * FROM clean_data"
df_stations = pd.read_sql(query, conn)

# Fermer la connexion
conn.close()

# Afficher les premières lignes des données pour s'assurer que tout est chargé correctement
df_stations.head()


,arrondissement_commune,longitude,latitude,nombre_places_disponibles,nombre_velos_disponibles,ville
0,Vitry-sur-Seine,2.396302,48.778193,10,10.0,Paris
1,Paris,2.337360,48.879296,16,4.0,Paris
2,Paris,2.343335,48.819428,33,24.0,Paris
3,Saint-Denis,2.358867,48.936269,3,16.0,Paris
4,Paris,2.353468,48.835093,39,7.0,Paris


## Visualisons à présent tout ca : nous créerons une interface intuitive et utile pour l'utilisateur, en effet il suffira de cliquer le nom de la ville pour avoir accès à toutes les stations de la ville en question et sur chaque point de stationnement, nous pouvons cliquer dessus pour avoir accès à des informations.

In [29]:
import folium
from ipywidgets import interact, widgets
from IPython.display import display

# Fonction pour afficher la carte en fonction de la ville sélectionnée
def afficher_carte(ville_selectionnee):
    # Filtrer les données pour la ville sélectionnée
    df_ville = df_stations[df_stations['ville'] == ville_selectionnee]
    
    
    # Créer une carte centrée sur la première station
    lat_centre = df_ville.iloc[0]['latitude']
    lon_centre = df_ville.iloc[0]['longitude']
    m = folium.Map(location=[lat_centre, lon_centre], zoom_start=13)
    
    # Ajouter des marqueurs pour chaque station
    for _, row in df_ville.iterrows():
        popup_info = (
            f"Station : {row['arrondissement_commune']}<br>"
            f"Places disponibles : {row['nombre_places_disponibles']}<br>"
            f"Vélos disponibles : {row['nombre_velos_disponibles']}<br>"
        )
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=popup_info,
            tooltip="Cliquez pour plus d'infos"
        ).add_to(m)
    
    # Afficher la carte
    display(m)

# Obtenir la liste unique des villes
villes_disponibles = df_stations['ville'].unique()

# Créer un widget interactif pour sélectionner la ville
interact(afficher_carte, ville_selectionnee=widgets.Dropdown(options=villes_disponibles, description="Ville"))


interactive(children=(Dropdown(description='Ville', options=('Paris', 'Toulouse', 'Lille'), value='Paris'), Ou…

<function __main__.afficher_carte(ville_selectionnee)>

## Visualisons la carte et enregistrons la sur format html si necessaire.

In [30]:
# Fonction pour afficher la carte en fonction de la ville sélectionnée
def afficher_carte(ville_selectionnee, enregistrer=False):
    # Filtrer les données pour la ville sélectionnée
    df_ville = df_stations[df_stations['ville'] == ville_selectionnee]
    
    # Créer une carte centrée sur la première station
    if not df_ville.empty:
        lat_centre = df_ville.iloc[0]['latitude']
        lon_centre = df_ville.iloc[0]['longitude']
        m = folium.Map(location=[lat_centre, lon_centre], zoom_start=13)
    else:
        print("Aucune donnée pour la ville sélectionnée.")
        return
    
    # Ajouter des marqueurs pour chaque station
    for _, row in df_ville.iterrows():
        popup_info = (
            f"Station : {row['arrondissement_commune']}<br>"
            f"Places disponibles : {row['nombre_places_disponibles']}<br>"
            f"Vélos disponibles : {row['nombre_velos_disponibles']}<br>"
        )
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=popup_info,
            tooltip="Cliquez pour plus d'infos"
        ).add_to(m)
    
    # Enregistrer la carte si demandé
    if enregistrer:
        file_name = f"Carte_{ville_selectionnee}.html"
        m.save(file_name)
        print(f"La carte a été enregistrée sous le nom : {file_name}.")
    
    # Afficher la carte
    display(m)

# Obtenir la liste unique des villes
villes_disponibles = df_stations['ville'].unique()

# Widget interactif pour sélectionner la ville et activer l'enregistrement
ville_widget = widgets.Dropdown(options=villes_disponibles, description="Ville")
enregistrer_widget = widgets.Checkbox(value=False, description="Enregistrer la carte en HTML")

# Fonction pour passer les valeurs des widgets à la fonction principale
def mise_a_jour_carte(ville_selectionnee, enregistrer):
    afficher_carte(ville_selectionnee, enregistrer)

# Créer une interface utilisateur interactive
interact(mise_a_jour_carte, ville_selectionnee=ville_widget, enregistrer=enregistrer_widget)


interactive(children=(Dropdown(description='Ville', options=('Paris', 'Toulouse', 'Lille'), value='Paris'), Ch…

<function __main__.mise_a_jour_carte(ville_selectionnee, enregistrer)>

# Effectuons à présent un filtrage des données 
L'utilisateur sélectionne une ville et saisit ses coordonnées.

Une carte s'affiche montrant :

    La position de l'utilisateur (marqueur rouge).
    
    Les stations dans un rayon de 1 km (marqueurs interactifs).
    
Chaque marqueur de station affiche les détails (nom, places disponibles, vélos disponibles) lorsqu'on clique dessus.

In [31]:
import folium
from ipywidgets import interact, widgets
from IPython.display import display
from math import radians, sin, cos, sqrt, atan2

# Fonction pour calculer la distance haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Fonction pour afficher la carte avec filtrage
def afficher_carte_filtrage(ville_selectionnee, user_lat, user_lon):
    # Filtrer les données pour la ville sélectionnée
    df_ville = df_stations[df_stations['ville'] == ville_selectionnee]
    
    # Ajouter une colonne "distance" dans le DataFrame
    df_ville['distance'] = df_ville.apply(
        lambda row: haversine(user_lat, user_lon, row['latitude'], row['longitude']), axis=1
    )
    
    # Filtrer les stations situées dans un rayon de 1 km
    df_filtre = df_ville[df_ville['distance'] <= 1]
    
    if df_filtre.empty:
        print("Aucune station disponible dans un rayon de 1 km.")
        return
    
    # Créer une carte centrée sur la position de l'utilisateur
    m = folium.Map(location=[user_lat, user_lon], zoom_start=15)
    
    # Ajouter un marqueur pour la position de l'utilisateur
    folium.Marker(
        location=[user_lat, user_lon],
        popup="Votre position",
        icon=folium.Icon(color="red")
    ).add_to(m)
    
    # Ajouter des marqueurs pour les stations filtrées
    for _, row in df_filtre.iterrows():
        popup_info = (
            f"Station : {row['arrondissement_commune']}<br>"
            f"Places disponibles : {row['nombre_places_disponibles']}<br>"
            f"Vélos disponibles : {row['nombre_velos_disponibles']}<br>"
        )
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=popup_info,
            tooltip="Cliquez pour plus d'infos"
        ).add_to(m)
    
    # Afficher la carte
    display(m)

# Obtenir la liste unique des villes
villes_disponibles = df_stations['ville'].unique()

# Widgets interactifs pour la ville et la position de l'utilisateur
ville_widget = widgets.Dropdown(options=villes_disponibles, description="Ville")
lat_widget = widgets.FloatText(description="Votre latitude", value=48.81)
lon_widget = widgets.FloatText(description="Votre longitude", value=2.34)

# Interface utilisateur interactive
interact(
    afficher_carte_filtrage,
    ville_selectionnee=ville_widget,
    user_lat=lat_widget,
    user_lon=lon_widget
)


interactive(children=(Dropdown(description='Ville', options=('Paris', 'Toulouse', 'Lille'), value='Paris'), Fl…

<function __main__.afficher_carte_filtrage(ville_selectionnee, user_lat, user_lon)>

## Améliorons le code pour permettre à ce que l'utilisateur puisse saisir son adresse simplement plutot que d'aller chercher ses coordonnées géographiques sur internet, l'application se chargera de calculer les coordonnées géographiques à sa place, cela rendra cela plus intuitif.

In [32]:
import folium
from ipywidgets import interact, widgets
from IPython.display import display
from math import radians, sin, cos, sqrt, atan2
import requests  # Pour les requêtes HTTP à l'API de géocodage

# Fonction pour calculer la distance haversine
def haversine(lat1, lon1, lat2, lon2):
    R = 6371  # Rayon de la Terre en km
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# Fonction pour convertir une adresse en latitude et longitude
def geocode_address(adresse):
    """Utilise OpenCage Geocoder pour obtenir la latitude et la longitude."""
    api_key = "ce9ef8e9c3c84797baa3ba25164b1674"  
    url = f"https://api.opencagedata.com/geocode/v1/json?q={adresse}&key={api_key}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        if data['results']:
            # Extraire la première correspondance
            latitude = data['results'][0]['geometry']['lat']
            longitude = data['results'][0]['geometry']['lng']
            return latitude, longitude
        else:
            print("Aucune correspondance trouvée pour cette adresse.")
            return None, None
    else:
        print(f"Erreur de géocodage : {response.status_code}")
        return None, None

# Fonction pour afficher la carte avec filtrage
def afficher_carte_filtrage(ville_selectionnee, adresse):
    # Convertir l'adresse en latitude et longitude
    user_lat, user_lon = geocode_address(adresse)
    
    if user_lat is None or user_lon is None:
        print("Impossible de localiser l'adresse. Veuillez vérifier votre saisie.")
        return

    # Filtrer les données pour la ville sélectionnée
    df_ville = df_stations[df_stations['ville'] == ville_selectionnee]
    
    # Ajouter une colonne "distance" dans le DataFrame
    df_ville['distance'] = df_ville.apply(
        lambda row: haversine(user_lat, user_lon, row['latitude'], row['longitude']), axis=1
    )
    
    # Filtrer les stations situées dans un rayon de 1 km
    df_filtre = df_ville[df_ville['distance'] <= 1]
    
    if df_filtre.empty:
        print("Aucune station disponible dans un rayon de 1 km.")
        return
    
    # Créer une carte centrée sur la position de l'utilisateur
    m = folium.Map(location=[user_lat, user_lon], zoom_start=15)
    
    # Ajouter un marqueur pour la position de l'utilisateur
    folium.Marker(
        location=[user_lat, user_lon],
        popup="Votre position",
        icon=folium.Icon(color="red")
    ).add_to(m)
    
    # Ajouter des marqueurs pour les stations filtrées
    for _, row in df_filtre.iterrows():
        popup_info = (
            f"Station : {row['arrondissement_commune']}<br>"
            f"Places disponibles : {row['nombre_places_disponibles']}<br>"
            f"Vélos disponibles : {row['nombre_velos_disponibles']}<br>"
        )
        folium.Marker(
            location=[row['latitude'], row['longitude']],
            popup=popup_info,
            tooltip="Cliquez pour plus d'infos"
        ).add_to(m)
    
    # Afficher la carte
    display(m)

# Obtenir la liste unique des villes
villes_disponibles = df_stations['ville'].unique()

# Widgets interactifs pour la ville et l'adresse
ville_widget = widgets.Dropdown(options=villes_disponibles, description="Ville")
adresse_widget = widgets.Text(description="Adresse", placeholder="Entrez votre adresse")

# Interface utilisateur interactive
interact(
    afficher_carte_filtrage,
    ville_selectionnee=ville_widget,
    adresse=adresse_widget
)


interactive(children=(Dropdown(description='Ville', options=('Paris', 'Toulouse', 'Lille'), value='Paris'), Te…

<function __main__.afficher_carte_filtrage(ville_selectionnee, adresse)>